## Boosting a decision stump
purpose: implementing boosting module with numpy and pandas library

# 1. Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Loading the data
pandas file parsing functions

1) read_csv : loading csv file with ',' separator

2) read_table : loading text file with '\t' separator(default)

3) read_fwf : loading text file with fixed width

4) read_clipboard : when copy the table from the web

In [16]:
loans = pd.read_csv('lending-club-data.csv')

In [17]:
print(type(loans)) # Dataframe 
print(loans.head(10)) # show the first 10 rows of the file, default = 5

<class 'pandas.core.frame.DataFrame'>
        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000             4975   36 months   
1  1077430    1314167       2500         2500             2500   60 months   
2  1077175    1313524       2400         2400             2400   36 months   
3  1076863    1277178      10000        10000            10000   36 months   
4  1075269    1311441       5000         5000             5000   36 months   
5  1072053    1288686       3000         3000             3000   36 months   
6  1071795    1306957       5600         5600             5600   60 months   
7  1071570    1306721       5375         5375             5350   60 months   
8  1070078    1305201       6500         6500             6500   60 months   
9  1069908    1305008      12000        12000            12000   36 months   

   int_rate  installment grade sub_grade          ...          sub_grade_num  \
0     10.65       162.8

In [18]:
type(loans)

pandas.core.frame.DataFrame

In [20]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1) # Make a safe_loans column 
loans.drop(['bad_loans'], axis =1) # Remove the bad_loans column
target = 'safe_loans'
loans = loans[features + [target]] # 

### Subsample dataset to make sure classes are balanced --> solve the imbalanced data problem

In [22]:
np.random.seed(1) # Assign the seed

safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac = percentage) # sampling in the datafame with fraction
loans_data = risky_loans_raw.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


### Transform categorical data into binary features

In this assignment, we will work with binary decision trees

In [25]:
loans_data = risky_loans.append(safe_loans)
feature = features[1]
print(feature)
loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})
loans_data_one_hot_encoded.head()

term


1     {u' 60 months': 1}
6     {u' 60 months': 1}
7     {u' 60 months': 1}
10    {u' 36 months': 1}
12    {u' 36 months': 1}
Name: term, dtype: object

In [27]:
type(loans_data_one_hot_encoded)

pandas.core.series.Series